In [ ]:
import matplotlib.pyplot as plt
import io
from PIL import Image, ImageChops

white = (255, 255, 255, 255)

def latex_to_img(tex):
    buf = io.BytesIO()
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.axis('off')
    plt.text(0.05, 0.5, f'${tex}$', size=40)
    plt.savefig(buf, format='png')
    plt.close()

    im = Image.open(buf)
    bg = Image.new(im.mode, im.size, white)
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    return im.crop(bbox)

In [ ]:
from scipy.stats import wilcoxon
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.5f}'.format

def wilcoxon_test(results_best,results_compare):
    pvalue = wilcoxon(x=results_best,y=results_compare).pvalue
    if pvalue<=0.001:
        return {}
    elif pvalue>0.001 and pvalue<0.05:
        return {'dag':'--rwrap'}
    else:
        return {'ddag':'--rwrap'}

In [ ]:
import pandas as pd
import os
from functools import partial

export_dir = '/home/pgonzalez/Dropbox/tnnlshistnet'

path = "results"

def show_results_table(experiment_names,base_path='', include_std=False, row_names=None,error_measures=['AE','RAE']):
    table = pd.DataFrame(columns=error_measures,dtype='float')
    results_error = {}
    for error_measure in error_measures:
        results_error[error_measure] = dict()


    for experiment in experiment_names:
        results = pd.read_csv(os.path.join(base_path,path,experiment+'_errors.txt'))
        for error_measure in error_measures:
            results_error[error_measure][experiment] =results[error_measure]
            table.loc[experiment,error_measure] = results[error_measure].mean()

    #table = table.style.highlight_min(color = 'yellow', axis = 0)

    #get best method for ae and best method for rae
    best_method = {}
    for error_measure in error_measures:
        best_method[error_measure] = table.index[table[error_measure].argmin()]

    table_style = table.style

    #make wixcolom testing MAE
    for experiment in experiment_names:
        for error_measure in error_measures:
            if experiment!=best_method[error_measure]:
                table_style.set_properties(subset=(experiment,error_measure),**wilcoxon_test(results_error[error_measure][experiment],results_error[error_measure][best_method[error_measure]]))
        
    for error_measure in error_measures:    
        table_style = table_style.highlight_min(axis=0,props='textbf:--rwrap;',subset=error_measure)
    #table = table.style.highlight_min(color = 'yellow', axis = 0)

    def add_deviation(x,std):
        return "{:.5f}".format(x)+' $\pm$ ' +"{:.5f}".format(std)

    use_f = lambda x: partial(add_deviation, std=x)
    formatters = [add_deviation]

    if include_std:
        for experiment_name in experiment_names:
            for error_measure in error_measures:
                std_value = results_error[error_measure][experiment_name].std()
                table_style = table_style.format(formatter=use_f(std_value),subset=(experiment_name,error_measure))
            

    latex_code = table_style.to_latex(hrules=True,column_format="r|"+"r"*len(table.columns))
    if row_names is not None:
        for experiment_name,row_name in zip(experiment_names,row_names):
            latex_code = latex_code.replace(experiment_name,row_name)

    return latex_code.replace("_","\\_"), results_error

## Número de parámetros de cada red

In [ ]:
import torch
def get_num_parameters(state_dict):
    return sum(p.numel() for p in state_dict.values())

#experiment_names = ["deepsets_avg_T1A","deepsets_max_T1A","deepsets_median_T1A","settransformers_T1A","histnet_sigmoid_T1A","histnet_hard_T1A","histnet_soft_T1A","histnet_softrbf_T1A","deepsets_avg_T1B","deepsets_max_T1B","deepsets_median_T1B","settransformers_T1B","histnet_sigmoid_T1B","histnet_hard_T1B","histnet_soft_T1B","histnet_softrbf_T1B"]
experiment_names = ["deepsets_avg_T1A","deepsets_max_T1A","deepsets_median_T1A","settransformers_T1A","histnet_sigmoid_T1A","histnet_hard_T1A","histnet_soft_T1A","histnet_softrbf_T1A"]
for experiment in experiment_names:
    #load the model
    print(experiment+":"+str(get_num_parameters(torch.load(os.path.join('savedmodels',experiment+'.pkl')))))


## Results T1A

In [ ]:
experiment_names = ["deepsets_avg_T1A","deepsets_max_T1A","deepsets_median_T1A","settransformers_T1A","histnet_hard_T1A"]
row_names = ["Deepsets (avg)","Deepsets (max)","Deepsets (median)","SetTransformers","HistNetQ"]
t1a_table,_ = show_results_table(experiment_names=experiment_names, base_path='', include_std=True,error_measures=['AE','RAE'], row_names=row_names)
print(t1a_table)
with open(os.path.join(export_dir,'tables/t1a_table.tex'),'w') as f:
    f.write(t1a_table)


## Results T1A (Ablation study)
Results without sample mixer

In [ ]:
experiment_names = ["deepsets_avg_T1A_ablation","deepsets_max_T1A_ablation","deepsets_median_T1A_ablation","settransformers_T1A_ablation","histnet_hard_T1A_ablation"]
row_names = ["Deepsets (avg)","Deepsets (max)","Deepsets (median)","SetTransformers","HistNetQ"]
t1a_table,_ = show_results_table(experiment_names=experiment_names, base_path='', include_std=True,error_measures=['AE','RAE'], row_names=row_names)
print(t1a_table)
with open(os.path.join(export_dir,'tables/t1a_ablation_table.tex'),'w') as f:
    f.write(t1a_table)

## Results T1B

In [ ]:
experiment_names = ["deepsets_avg_T1B","deepsets_max_T1B","deepsets_median_T1B","settransformers_T1B","histnet_hard_T1B_64bits"]
row_names = ["Deepsets (avg)","Deepsets (max)","Deepsets (median)","SetTransformers","HistNetQ (Hard 32 bins)","HistNetQ (Hard 64 bins)"]
t1b_table,_=show_results_table(experiment_names,include_std=True,row_names=row_names)
print(t1b_table)
with open(os.path.join(export_dir,'tables/t1b_table.tex'),'w') as f:
    f.write(t1b_table)

## Results T1B (Ablation study)


In [ ]:
experiment_names = ["deepsets_avg_T1B_ablation","deepsets_max_T1B_ablation","deepsets_median_T1B_ablation","settransformers_T1B_ablation","histnet_hard_T1B_64bins_ablation"]
row_names = ["Deepsets (avg)","Deepsets (max)","Deepsets (median)","SetTransformers","HistNetQ (Hard 64 bins)"]
t1b_table,_=show_results_table(experiment_names,include_std=True,row_names=row_names)
print(t1b_table)
with open(os.path.join(export_dir,'tables/t1b_table_ablation.tex'),'w') as f:
    f.write(t1b_table)

## Análisis por número de bins

In [ ]:
import numpy as np
import seaborn as sns
sns.set_theme()
experiment_names = ["histnet_hard_T1B_8bins","histnet_hard_T1B_16bins","histnet_hard_T1B","histnet_hard_T1B_64bins"]
row_names = ["HistNet (8 bins)","HistNet (16 bins)","HistNet (32 bins)","HistNet (64 bins)"]
bins_comp, results_error=show_results_table(experiment_names, row_names = row_names, include_std=True)
print(bins_comp)
with open(os.path.join(export_dir,'tables/bins_comp.tex'),'w') as f:
    f.write(bins_comp)

chart_data = pd.DataFrame(columns=("n_bins","rae"),dtype = ('int','float'))
chart_data = chart_data.append({"n_bins":8,"rae":results_error['RAE']['histnet_hard_T1B_8bins'].mean()},ignore_index=True)
chart_data = chart_data.append({"n_bins":16,"rae":results_error['RAE']['histnet_hard_T1B_16bins'].mean()},ignore_index=True)
chart_data = chart_data.append({"n_bins":32,"rae":results_error['RAE']['histnet_hard_T1B'].mean()},ignore_index=True)
chart_data = chart_data.append({"n_bins":64,"rae":results_error['RAE']['histnet_hard_T1B_64bins'].mean()},ignore_index=True)
chart_data.n_bins = chart_data.n_bins.astype(int)

plt.figure(figsize=(6,3))
sns.lineplot(data=chart_data,x='n_bins',y='rae',marker='^',markersize=10)
plt.xlabel("Number of bins")
plt.ylabel("RAE")
plt.title("Performance by number of bins")
plt.savefig(os.path.join(export_dir,'figures/bybins.pdf'), dpi=300, bbox_inches='tight')


## ISAB como feature extraction

In [ ]:
experiment_names = ["isab_deepsets_max_T1B","isab_deepsets_avg_T1B","isab_deepsets_median_T1B","isab_histnet_sigmoid_T1B","isab_histnet_soft_T1B","isab_histnet_softrbf_T1B","isab_histnet_hard_T1B"]
isab_comp=show_results_table(experiment_names)
print(isab_comp)
with open(os.path.join(export_dir,'tables/isab_as_fe.tex'),'w') as f:
    f.write(isab_comp)

## No fe extraction

In [ ]:
experiment_names = ["norff_settransformers_T1B","norff_deepsets_max_T1B","norff_deepsets_avg_T1B","norff_deepsets_median_T1B","norff_histnet_sigmoid_T1B","norff_histnet_soft_T1B","norff_histnet_softrbf_T1B","norff_histnet_hard_T1B"]
nofe_comp=show_results_table(experiment_names)
print(nofe_comp)
with open(os.path.join(export_dir,'tables/nofe.tex'),'w') as f:
    f.write(nofe_comp)

## Resultados en el dataset MNIST
Básicamente este experimento consiste en un APP sobre el conjunto de test con 5000 muestras de prueba.

In [ ]:
experiment_names = ["settransformers_mnist","deepsets_max_mnist","deepsets_avg_mnist","deepsets_median_mnist","histnet_softrbf_mnist","histnet_hard_mnist","CC_mnist","AC_mnist","PAC_mnist","HDy_mnist","EM_mnist"]
mnist_results=show_results_table(experiment_names,base_path='cifar_mnist')
print(mnist_results)
with open(os.path.join(export_dir,'tables/mnist.tex'),'w') as f:
    f.write(mnist_results)

## Resultados CIFAR 10

In [ ]:
experiment_names = ["settransformers_ae_pretrained_cifar10","deepsets_avg_ae_pretrained_cifar10","histnet_hard_ae_pretrained_cifar10","CC_cifar10","AC_cifar10","PAC_cifar10","HDy_cifar10","EM_cifar10","EM-NBVS_cifar10","EM-BCTS_cifar10","EM-VS_cifar10"]
#,
cifar10_results=show_results_table(experiment_names,base_path="cifar_mnist",add_mse=True)
print(cifar10_results)
with open(os.path.join(export_dir,'tables/cifar10.tex'),'w') as f:
    f.write(cifar10_results)

## Resultados FASHIONMNIST

In [ ]:
experiment_names = ["settransformers_ae_fashionmnist","settransformers_rae_fashionmnist","settransformers_mse_fashionmnist",
"deepsets_avg_ae_fashionmnist","deepsets_avg_ae_fashionmnist","deepsets_avg_rae_fashionmnist","deepsets_avg_mse_fashionmnist",
"deepsets_median_ae_fashionmnist","deepsets_median_rae_fashionmnist","deepsets_median_mse_fashionmnist","deepsets_max_ae_fashionmnist",
"deepsets_max_rae_fashionmnist","deepsets_max_mse_fashionmnist",
"histnet_hard_ae_fashionmnist","histnet_hard_rae_fashionmnist","histnet_hard_mse_fashionmnist",
#"histnet_hard_batched_ae_fashionmnist","histnet_hard_batched_rae_fashionmnist","histnet_hard_batched_mse_fashionmnist",
"CC_fashionmnist","PCC_fashionmnist","AC_fashionmnist","PAC_fashionmnist","HDy_fashionmnist",
"EM_fashionmnist","EM-NBVS_fashionmnist","EM-BCTS_fashionmnist","EM-VS_fashionmnist"]
#,

fashionmnist_results,_=show_results_table(experiment_names,base_path="cifar_mnist",include_std=True,error_measures=['AE','RAE','MSE'])
print(fashionmnist_results)
with open(os.path.join(export_dir,'tables/fashionmnist.tex'),'w') as f:
    f.write(fashionmnist_results)

## Fashion-mnist resumida
### AE


In [ ]:
experiment_names = ["settransformers_ae_fashionmnist","deepsets_avg_ae_fashionmnist","deepsets_median_ae_fashionmnist","deepsets_max_ae_fashionmnist","histnet_hard_ae_fashionmnist",
"CC_fashionmnist","PCC_fashionmnist","AC_fashionmnist","PAC_fashionmnist","HDy_fashionmnist",
"EM_fashionmnist","EM-NBVS_fashionmnist","EM-BCTS_fashionmnist","EM-VS_fashionmnist"]
row_names  = ["SetTransformers","Deepsets (avg)","Deepsets (median)","Deepsets (max)","HistNet",
"CC","PCC","ACC","PACC","HDy","EM","EM-NBVS","EM-BCTS","EM-VS"]
#,
fashionmnist_results,_=show_results_table(experiment_names,row_names = row_names, base_path="cifar_mnist",error_measures=['AE'])
print(fashionmnist_results)

experiment_names = ["settransformers_rae_fashionmnist","deepsets_avg_rae_fashionmnist","deepsets_median_rae_fashionmnist","deepsets_max_rae_fashionmnist","histnet_hard_rae_fashionmnist",
"CC_fashionmnist","PCC_fashionmnist","AC_fashionmnist","PAC_fashionmnist","HDy_fashionmnist",
"EM_fashionmnist","EM-NBVS_fashionmnist","EM-BCTS_fashionmnist","EM-VS_fashionmnist"]
fashionmnist_results,_=show_results_table(experiment_names, row_names = row_names, base_path="cifar_mnist",error_measures=['RAE'])
print(fashionmnist_results)

experiment_names = ["settransformers_mse_fashionmnist","deepsets_avg_mse_fashionmnist","deepsets_median_mse_fashionmnist","deepsets_max_mse_fashionmnist","histnet_hard_mse_fashionmnist",
"CC_fashionmnist","PCC_fashionmnist","AC_fashionmnist","PAC_fashionmnist","HDy_fashionmnist",
"EM_fashionmnist","EM-NBVS_fashionmnist","EM-BCTS_fashionmnist","EM-VS_fashionmnist"]
fashionmnist_results,_=show_results_table(experiment_names, row_names = row_names, base_path="cifar_mnist",error_measures=['MSE'])
print(fashionmnist_results)

## Optimizando diferentes medidas de error

In [ ]:
experiment_names = ["histnet_hard_ae_pretrained_cifar10","histnet_hard_rae_pretrained_cifar10","histnet_hard_mse_pretrained_cifar10"]
#,
cifar10_results=show_results_table(experiment_names,base_path="cifar_mnist",add_mse=True)
print(cifar10_results)
with open(os.path.join(export_dir,'tables/cifar10.tex'),'w') as f:
    f.write(cifar10_results)

## Comparación con baselines la competición LEQUA

In [ ]:
from dlquantification.utils.lossfunc import MRAE
import torch

def compute_errors(baselines,baseline_path,test_prevalences,dataset,loss_mrae):
    test_prevalences = pd.read_csv(test_prevalences)
    for baseline in baselines:
        export_path = os.path.join("results/", baseline + "_"+dataset+"_errors.txt")
        if not os.path.exists(export_path):
            errors = pd.DataFrame(columns=('AE','RAE'),index=range(5000))
            results = pd.read_csv(os.path.join(baseline_path,baseline)+'.csv')
            for i in range(len(results)):
                errors.iloc[i]['RAE']=loss_mrae(torch.FloatTensor(test_prevalences.iloc[i,1:].to_numpy()), torch.FloatTensor(results.iloc[i,1:].to_numpy())).numpy()
                errors.iloc[i]['AE']=torch.nn.functional.l1_loss(torch.FloatTensor(test_prevalences.iloc[i,1:].to_numpy()), torch.FloatTensor(results.iloc[i,1:].to_numpy())).numpy()
            errors.to_csv(export_path, index_label="id")


baselines=['CC','ACC','HDy','PACC','PCC','QuaNet','SLD']
#For this experiments we need to compute the errors as we only have the predictions

loss_mrae = MRAE(eps=1.0 / (2 * 250), n_classes=2).MRAE
baseline_path_T1A = os.path.join(path,'codalab/T1A')
test_prevalences = './lequa/T1A/public/test_prevalences.txt'
compute_errors(baselines,baseline_path_T1A,test_prevalences,'T1A',loss_mrae)
baselines=['CC','ACC','PACC','PCC','SLD']
loss_mrae = MRAE(eps=1.0 / (2 * 1000), n_classes=28).MRAE
baseline_path_T1B = os.path.join(path,'codalab/T1B')
test_prevalences = './lequa/T1B/public/test_prevalences.txt'
compute_errors(baselines,baseline_path_T1B,test_prevalences,'T1B',loss_mrae)

In [ ]:
experiment_names = ["CC_T1A","PCC_T1A","ACC_T1A","PACC_T1A","HDy_T1A","QuaNet_T1A","SLD_T1A","histnet_hard_T1A"]
baseline_results=show_results_table(experiment_names)
print(baseline_results)
with open(os.path.join(export_dir,'tables/baselines_t1a.tex'),'w') as f:
    f.write(baseline_results)

In [ ]:
experiment_names = ["CC_T1B","PCC_T1B","ACC_T1B","PACC_T1B","SLD_T1B","histnet_hard_T1B"]
baseline_results=show_results_table(experiment_names)
print(baseline_results)
with open(os.path.join(export_dir,'tables/baselines_t1b.tex'),'w') as f:
    f.write(baseline_results)